In [1]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import wandb
import functools

from src.display import showarray
from src.mask import MaskGenerator
from src.datagen import DatasetFillGenerator
from src.augmenters import identical_augmenter, mask_image_augmenter
from src.builders.unet import UNETBuilder

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
config = {
  "learning_rate": 0.001,
  "epochs": 16,
  "batch_size": 16,
  "mask_gen_degree": "HEAVY",
  "mask_gen_min_width": 3,
  "mask_gen_max_width": 9,
  "learning_rate": 0.001
}
model_params = {
  "n_filters": 32,
  "n_layers": 3,
  "n_convs": 1,
  "activation": "elu",
  "dropout": 0.4,
}
config.update({f"unet_{key}": val for key, val in model_params.items()})
wandb.init(project="cv3-ii-ae-unet", entity="put_dl_team", config=config)
# wandb.config.update(config)


IMAGE_SIZE = (256, 256)
CHANNELS = 3
BATCH_SIZE = wandb.config["batch_size"]

MASK_GEN_PARAM = {
    "degree": wandb.config["mask_gen_degree"],
    "min_width": wandb.config["mask_gen_min_width"],
    "max_width": wandb.config["mask_gen_max_width"],
}

mask_generator = MaskGenerator(*IMAGE_SIZE, CHANNELS, **MASK_GEN_PARAM)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sbartekt (put_dl_team). Use `wandb login --relogin` to force relogin


In [4]:
def scale(x):
    return x / 255.0

def recast_to_image(tensor: tf.Tensor) -> np.ndarray:
    return tf.cast(tensor * 255, tf.uint8).numpy()


ds_train, ds_valid = tf.keras.preprocessing.image_dataset_from_directory(
    directory="../data/1-8size", label_mode=None, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
    shuffle=True, seed=42, validation_split=0.3, subset="both",
)
ds_train = ds_train.map(scale)
ds_valid = ds_valid.map(scale)

for batch in ds_valid.take(1):
    showcase_images = batch[:5]


class ImageFillCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, showcase_images, mask_generator):
        self.model = model
        self.showcase_images = showcase_images
        self.masked_images = mask_image_augmenter(self.showcase_images, mask_generator, 1.0)

    def on_epoch_end(self, epoch, logs=None):
        nn_filled = self.model.predict(self.masked_images)
        all_joint = []
        for i in range(5):
            masked = recast_to_image(self.masked_images[i])
            original_image = recast_to_image(self.showcase_images[i])
            filled_image = recast_to_image(nn_filled[i])
            joint = np.concatenate([masked, original_image, filled_image], axis=1)
            all_joint.append(joint)
        all_joint = np.concatenate(all_joint, axis=0)
        wandb.log({"sample_fill": wandb.Image(all_joint)})

dataset_image_augmenter = functools.partial(mask_image_augmenter, mask_generator=mask_generator, max_val=1.0)

np.random.seed(42)
train_generator = DatasetFillGenerator(ds_train, IMAGE_SIZE, CHANNELS, dataset_image_augmenter)
valid_generator = DatasetFillGenerator(ds_valid, IMAGE_SIZE, CHANNELS, dataset_image_augmenter)

Found 2188 files belonging to 1 classes.
Using 1532 files for training.
Using 656 files for validation.


In [5]:
IM_SHAPE = (*IMAGE_SIZE, CHANNELS)
builder = UNETBuilder(IM_SHAPE, IM_SHAPE, **model_params)
model = builder.build()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 128, 128, 32  0           ['conv2d[0][0]']                 
                                )                                                             

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=wandb.config["learning_rate"]),
    loss=tf.keras.losses.MeanSquaredError()
)

In [7]:
model.fit(train_generator, epochs=10, validation_data=valid_generator, callbacks=[wandb.keras.WandbCallback(), ImageFillCallback(model, showcase_images, mask_generator)])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
96/96 [==============================] - ETA: 0s - loss: 0.0286

INFO:tensorflow:Assets written to: c:\Users\Bartosz\PycharmProjects\CV3_PROJECT\cv-image-inpainting\notebooks\wandb\run-20230119_211940-fz5rfjip\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\Bartosz\PycharmProjects\CV3_PROJECT\cv-image-inpainting\notebooks\wandb\run-20230119_211940-fz5rfjip\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\Bartosz\PycharmProjects\CV3_PROJECT\cv-image-inpainting\notebooks\wandb\run-20230119_211940-fz5rfjip\files\model-best)... Done. 0.0s


96/96 [==============================] - 91s 896ms/step - loss: 0.0286 - val_loss: 0.0132
Epoch 2/10
96/96 [==============================] - ETA: 0s - loss: 0.0109

...

96/96 [==============================] - 132s 1s/step - loss: 0.0068 - val_loss: 0.0049
Epoch 4/10
96/96 [==============================] - ETA: 0s - loss: 0.0047

INFO:tensorflow:Assets written to: c:\Users\Bartosz\PycharmProjects\CV3_PROJECT\cv-image-inpainting\notebooks\wandb\run-20230119_211940-fz5rfjip\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\Bartosz\PycharmProjects\CV3_PROJECT\cv-image-inpainting\notebooks\wandb\run-20230119_211940-fz5rfjip\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\Bartosz\PycharmProjects\CV3_PROJECT\cv-image-inpainting\notebooks\wandb\run-20230119_211940-fz5rfjip\files\model-best)... Done. 0.1s
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


1/1 [==============================] - 0s 27ms/step


KeyboardInterrupt: 

In [ ]:
wandb.finish(0)

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▁▁▁▁▁▁
val_loss,█▄▃▂▂▂▁▁▁▁
best_epoch,9
best_val_loss,0.00301
epoch,9
loss,0.00308
val_loss,0.00301
